In [0]:
CREATE OR REFRESH STREAMING TABLE cdf_bronze2 AS
SELECT * FROM STREAM cdf_bronze_tbl

In [0]:
CREATE OR REFRESH STREAMING TABLE emp_silver_streaming
  COMMENT 'SCD Type 2 Historical Customer Data';

In [0]:
CREATE FLOW new_scd_type_2_flow AS 
AUTO CDC INTO emp_silver_streaming -- Target table to update with SCD Type 1 (or 2)
FROM STREAM  cdf_bronze2  -- Source records to determine updates, deletes and inserts
  KEYS (EmployeeID)                              -- Primary key for identifying records
  APPLY AS DELETE WHEN changetype = "delete"       -- Handle deletes from source to the target
  SEQUENCE BY commit_timestamp                  -- Defines order of operations for applying changes
  COLUMNS * EXCEPT (changetype, commit_timestamp)     -- Select columns and exclude metadata fields
  STORED AS SCD TYPE 2;      -- Use Slowly Changing Dimension Type 1 to update the target table (no historical information)